In [ ]:
from diffusers import StableDiffusionPipeline
import torch
#loading the model from hugging face
#we're going to use a pre-trained model that is already available
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id) #we use a pretrained model from hugging face
pipe = pipe.to('cpu') #we move the model to the cpu

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [2]:
from PIL import Image
import os
os.makedirs("generated_images",exist_ok=True)

In [3]:
prompts = [
    "blue sunset over ocean",
    "purple sunset over ocean",
    "pink sunset over ocean",
    "orange sunset over ocean",
    "red sunset over ocean"
]

In [4]:
for i, prompt in enumerate(prompts):
    image = pipe(prompt).images[0]
    image_path=f"generated_images/image_{i}.png"
    image.save(image_path)
#generating images and saving them

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [4]:
import os

In [15]:
import cv2
frame_size=(512,512)
fps=24
image_duration=2
transition_duration=1
#initializing the video writer
video_path="animation.mp4"
fourcc=cv2.VideoWriter_fourcc(*"mp4v")
video=cv2.VideoWriter(video_path, fourcc, fps, frame_size)

In [7]:
image_directory = "generated_images"
image_paths=[os.path.join(image_directory,f) for f in sorted(os.listdir(image_directory)) if f.endswith(".png")]
print(image_paths)

['generated_images/image_0.png', 'generated_images/image_1.png', 'generated_images/image_2.png', 'generated_images/image_3.png', 'generated_images/image_4.png']


In [16]:
len(image_paths)

5

function to read the image at given path and resize it to the given frame size

In [17]:
def read_resize(image_path, frame_size):
    img=cv2.imread(image_path)
    img=cv2.resize(img, frame_size)
    return img

adding images with crossfade transitions

In [18]:
for i in range(len(image_paths)-1):
    img1=read_resize(image_paths[i], frame_size)
    img2=read_resize(image_paths[i+1], frame_size)
    for _ in range(image_duration*fps):
        video.write(img1)
    for j in range(transition_duration*fps):
        alpha=j/fps
        blended_img=cv2.addWeighted(img1, 1-alpha, img2, alpha, 0)
        video.write(blended_img)

In [20]:
#adding the last image for the full duration
last_image=read_resize(image_paths[-1],frame_size)
for _ in range(image_duration*fps):
    video.write(last_image)

In [21]:
video.release()